In [1]:
import numpy as np
import pandas as pd
import glob
import os
from root_pandas import read_root

#from root_pandas import read_root
from IPython.display import clear_output

ModuleNotFoundError: No module named 'root_pandas'

In [2]:
# First is the folder name, second is the way how it should be treated (true, signal, data)
projects =    ["bnb_nu_cosmic_0s0t_dev",
               "bnb_nue_cosmic_0s0t_dev",
               "bnb_nu_cosmic_tune3_0s0t_dev",
               "bnb_nu_cosmic_overlaid_0s0t_dev",
               "bnb_nu_cosmic_inducedcharge",
               "bnb_nu_cosmic_lightbug",
               "cosmic_intime_0s0t_dev",
               "data_bnb_a_0s0t",
               "data_bnb_b_0s0t",
               "data_bnbext_a_0s0t",
              ]

project_out = ["nu",
               "nue",
               "nu_tune3",
               "nu_overlaid",
               "nu_induced",
               "nu_lightbug",
               "intime",
               "bnb_a",
               "bnb_b",
               "bnbext_a",
              ]


projects = ["data_bnbext_a_0s0t_v14","data_bnb_a_0s0t_v14","data_bnb_b_0s0t_v14"]
project_out = ["bnbext_a_v14", "bnb_a_v14", "bnb_b_v14"]

In [3]:
# Main datafram
columns= [ # True
          "run", "subrun", "event",
          "bnbweight","nu_E",
          "true_1eX_signal",
          #"nu_daughters_pdg","nu_daughters_E",
          "true_vx","true_vy","true_vz",
           # Reco
          "category",
          "n_primaries", 'primary_indexes',
          "flash_time", "flash_PE",
          "flash_x","flash_score","TPC_x",
          "flash_passed",
          "chargecenter_candidates_total",
          "number_primary_tracks", "number_primary_showers",
          "shower_passed","track_passed",
          "flash_PE_max", "flash_hypo_PE", "flash_matchid"
]


# Candidates dataframe
columns_candidates_flat = ["bnbweight","true_1eX_signal","true_vz","n_primaries", "nu_E",
                           "flash_y", "flash_sy", "flash_z", "flash_sz", "flash_PE_max"
                          ]

columns_candidates = ["number_tracks", "number_showers",
                      "number_primary_tracks", "number_primary_showers",
                      "shower_passed","track_passed",
                      "flash_passed",
                      "chargecenter_candidates_x",
                      "chargecenter_candidates_y",
                      "chargecenter_candidates_z",
                      "chargecenter_candidates_total"
                     ]

In [4]:
flux_corr = 1.028
mass_p= 0.93827 #GeV
mass_e= 0.00511 #GeV
# Minimum reconstructable energies:
min_e = 0.02+mass_e # 20MeV
min_p = 0.04+mass_p # 40MeV
min_pi = 0.17

# dead wires
z_dead_start = 675
z_dead_end=z_dead_start+100

In [5]:
for project,sample in zip(projects,project_out):  
    
    gpvmsdir = '/run/user/1002/gvfs/sftp:host=uboonegpvm02.fnal.gov,user=wvdp'
    inputlist = []
    inputlist += glob.glob("/home/wouter/Templates/July/" + project + "/*.root")
    filelist  = []
    for fname in inputlist:
        filelist.append(fname)
    print( "\n\nStarting project ",project,"with samplename ",sample, "Number of files: ", len(filelist))

    chunks = []
    chunks_candidate = []
    
    total_pot = 0
    total_events = 0
    total_weighted_events = 0
    
    for i_f,fname in enumerate(filelist):
        #clear_output(wait=True)
        print (i_f, end="\r")
        
         # Store the POT of the sample
        df_pot = read_root(fname, 'wouterNueCC/pot')
        total_pot += df_pot['pot'].sum()
        # Write this dataframe to a txtfile.
        df_pot[['run', 'subrun']].to_csv("./input/"+sample + '_run_subrun.txt', header=None, index=None, sep=' ', mode='a')
        
        # Load real data_frame
        dftemp=read_root(fname,"wouterNueCC/pandoratree",columns=columns)  
        total_events += len(dftemp.index)
        total_weighted_events += sum(dftemp["bnbweight"])
        dftemp_candidate = read_root(fname, 'wouterNueCC/pandoratree',
                                     columns=columns_candidates + columns_candidates_flat,
                                     flatten=columns_candidates)

        chunks.append(dftemp)
        chunks_candidate.append(dftemp_candidate)
    
    #clear_output(wait=True)
    print ("Done loading, start processing df")
        
    df = pd.concat(chunks,ignore_index=True,copy=False) 
    df["true_z_deadwire"] = ~df['true_vz'].between(z_dead_start,z_dead_end)
    #df["true_1e1p_signal"] = df.apply(true_thresholds_1e1p,axis=1)
    #df.drop(["nu_daughters_pdg","nu_daughters_E"],inplace=True,axis=1)
    df["bnbweight"]*=flux_corr
    df["passed_primaries"] = df["flash_passed"].apply(lambda x: sum(x>-2))
    df.to_pickle("./input/df_"+sample+"_flash.pckl")
    
    
    print ("Done, start processing df_candidate")
    df_candidate = pd.concat(chunks_candidate,ignore_index=True,copy=False) 
    df_candidate["true_z_deadwire"] = ~df_candidate['true_vz'].between(z_dead_start,z_dead_end)
    # Candidate is good if:
    df_candidate.eval('good = (true_1eX_signal==1) & (number_primary_tracks==track_passed) \
                       & (number_primary_showers==shower_passed) & (true_z_deadwire==1) \
                       & (chargecenter_candidates_total>1)', inplace=True)
    
    df_candidate.eval('bad = (track_passed==0) & (shower_passed==0) & (chargecenter_candidates_total>1)', inplace=True)
    df_candidate["bnbweight"]*=flux_corr
    df_candidate.eval('diff_z = chargecenter_candidates_z - flash_z', inplace=True)
    df_candidate.eval('diff_sz = diff_z/flash_sz', inplace=True)
    df_candidate.eval('diff_y = chargecenter_candidates_y - flash_y', inplace=True)
    df_candidate.eval('diff_sy = diff_y/flash_sy', inplace=True)
    df_candidate.eval('charge_ratio = chargecenter_candidates_total/flash_PE_max', inplace=True)
    df_candidate.to_pickle("./input/df_"+sample+"_flash_candidate.pckl")

    print("Done processing")
    print("Total events:", total_events)
    print("Total weighted events:", int((total_weighted_events*1000))/1000)
    print("total_pot:", total_pot)



Starting project  data_bnbext_a_0s0t_v14 with samplename  bnbext_a_v14 Number of files:  460
Done loading, start processing df
Done, start processing df_candidate
Done processing
Total events: 286339
Total weighted events: -614908320284672.0
total_pot: 104587999.309


Starting project  data_bnb_a_0s0t_v14 with samplename  bnb_a_v14 Number of files:  224
Done loading, start processing df
Done, start processing df_candidate
Done processing
Total events: 170825
Total weighted events: -366843894169600.0
total_pot: 34270451.0572


Starting project  data_bnb_b_0s0t_v14 with samplename  bnb_b_v14 Number of files:  16
Done loading, start processing df
Done, start processing df_candidate
Done processing
Total events: 778
Total weighted events: -1670742278144.0
total_pot: 0.0


In [ ]:
# Convert pickle to new version pandas:

inputlist = glob.glob("./input/*v14*.pckl")
#inputlist = ["./input/df_bnbext_a_flash_candidate.pckl","./input/df_bnbext_a_flash.pckl"]
for file in inputlist:
    print(file)
    df_nu = pd.read_pickle(file)
    df_nu.to_pickle(file)

./input/df_bnbext_a_flash_candidate.pckl
./input/df_bnbext_a_flash.pckl
